## Comprehensive example of Avalanche usage 

In [2]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.classic import SplitMNIST
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, \
     loss_metrics, timing_metrics, cpu_usage_metrics, confusion_matrix_metrics, disk_usage_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training.strategies import Naive

scenario = SplitMNIST(n_experiences=5)

# MODEL CREATION
model = SimpleMLP(num_classes=scenario.n_classes)

# DEFINE THE EVALUATION PLUGIN
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(num_classes=scenario.n_classes, save_image=False, stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger]
)

# CREATE THE STRATEGY INSTANCE (NAIVE)
cl_strategy = Naive(
    model, SGD(model.parameters(), lr=0.001, momentum=0.9),
    CrossEntropyLoss(), train_mb_size=500, train_epochs=1, eval_mb_size=100,
    evaluator=eval_plugin)

# TRAINING LOOP
print('Starting experiment...')
results = []

for experience in scenario.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(scenario.test_stream))

Starting experiment...
Start of experience:  0
Current Classes:  [1, 6]
-- >> Start of training phase << --
-- Starting training on experience 0 (Task 0) from train stream --
100%|███████████████████████████████████████████| 26/26 [00:03<00:00,  8.14it/s]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 67011.9424
	DiskUsage_MB/train_phase/train_stream/Task000 = 67011.9424
	Loss_Epoch/train_phase/train_stream/Task000 = 1.0213
	Loss_MB/train_phase/train_stream/Task000 = 0.1820
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0048
	Time_Epoch/train_phase/train_stream/Task000 = 3.1944
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8101
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.9875
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|███████████████████████████████████████████| 21/21 [00:00<00:00, 31.27it/s]
> Ev

Start of experience:  2
Current Classes:  [0, 7]
-- >> Start of training phase << --
-- Starting training on experience 2 (Task 0) from train stream --
100%|███████████████████████████████████████████| 25/25 [00:02<00:00,  8.67it/s]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 67051.0928
	DiskUsage_MB/train_phase/train_stream/Task000 = 67051.0928
	Loss_Epoch/train_phase/train_stream/Task000 = 1.8821
	Loss_MB/train_phase/train_stream/Task000 = 0.3261
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0052
	Time_Epoch/train_phase/train_stream/Task000 = 2.8851
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.5178
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.9681
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|███████████████████████████████████████████| 21/21 [00:00<00:00, 36.33it/s]
> Eval on experience 0 (Tas

	Loss_Stream/eval_phase/test_stream/Task000 = 2.1152
	StreamForgetting/eval_phase/test_stream = 0.7679
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.3179
Start of experience:  4
Current Classes:  [9, 5]
-- >> Start of training phase << --
-- Starting training on experience 4 (Task 0) from train stream --
100%|███████████████████████████████████████████| 23/23 [00:02<00:00,  8.55it/s]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 67090.4346
	DiskUsage_MB/train_phase/train_stream/Task000 = 67090.4346
	Loss_Epoch/train_phase/train_stream/Task000 = 2.4352
	Loss_MB/train_phase/train_stream/Task000 = 1.3918
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0053
	Time_Epoch/train_phase/train_stream/Task000 = 2.6901
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.2038
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.7568
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Startin